In [312]:
import pandas as pd
import numpy as np
import pickle
pd.set_option('display.max_columns', None)

from collections import Counter
from itertools import chain

import oracledb
connection = oracledb.connect(user="", password="", host="",port=,service_name="")

from tqdm import tqdm
tqdm.pandas()
import re
import os

In [6]:
import numpy
def coefficient_of_variation(data):
    mean=numpy.mean(data) #计算平均值
    std=numpy.std(data,ddof=0) #计算标准差
    cv=std/mean
    return cv
data_test_1=[1,2,3,4,5,6,7]
data_test_2=[1,1,1,4,7,7,7]
print('CV_1',coefficient_of_variation(data_test_1))
print('CV_2',coefficient_of_variation(data_test_2))


CV_1 0.5
CV_2 0.6943650748294136


In [101]:
def entropy(x):
    if len(x)==0:
        entropy=None
    else:
        dict_tem=Counter(x)
        list_ratio=[v/sum(dict_tem.values()) for v in dict_tem.values()]
        entropy=sum([-i*np.log2(i) for i in list_ratio])
    return entropy
entropy(['1','2','2','3','3','3','4','4','4','4'])

1.8464393446710154

In [181]:
# 补充文本内容
import textstat
from bs4 import BeautifulSoup

def calculate_metrics(text):
    pure_body_string_len = len(text)
    pure_body_lexicon_count = textstat.lexicon_count(text)
    pure_body_smog_index = textstat.smog_index(text)
    return pure_body_string_len, pure_body_lexicon_count, pure_body_smog_index


In [105]:
def gini(x):
    if len(x)==0:
        gini=None
    else:
        w = list(Counter(x).values())
        sampleCount = len(w)            #样本总数，即人数
        w.sort()                        #从小(穷)到大(富)排序
        cw = []
        for i in range(sampleCount):    #计算累积财富值
            cw.append(sum(w[:i+1]))     #cw[i]表示社会最穷的i+1个人的总财富

        totalWealth = cw[-1]            #社会总财富
        for i in range(len(cw)):        #计算累积财富占比
            cw[i] = cw[i] / totalWealth #cw[i]表示社会最穷的i+1个人的财富占比
        cw = [0] + cw                   #加0: 0%的人拥有0%的财富
        cp = [0] + [i/sampleCount for i in range(1,sampleCount+1)]

        B = 0                           #梯形法累加计算B区域面积
        for i in range(1,sampleCount+1):
            B = B + (cw[i] + cw[i-1]) * (cp[i] - cp[i-1]) / 2
        A = 0.5 - B                     #A+B = 0.5
        gini=A/(A+B)
    return gini
gini(['1','2','2','3','3','3','4','4','4','4'])

0.25

In [7]:
def calculate_rd_expenses_dynamic(df, columns, new_column_name):
    # 尝试将指定列转换为float类型
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    def row_calculator(row):
        values = [row[col] for col in columns if col in row]
        valid_values = [v for v in values if pd.notna(v)]
        return np.nan if len(valid_values) == 0 else np.mean(valid_values)
    
    df[new_column_name] = df.apply(row_calculator, axis=1)
    return df

# 公司论文数据

In [3]:
sql_string='select t.* from "上市公司匹配patstat_2" t'
df_firm_patstat = pd.read_sql(sql=sql_string, con=connection)
df_firm_patstat

/tmp/ipykernel_101377/3135114047.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_firm_patstat = pd.read_sql(sql=sql_string, con=connection)


,psn_id,psn_name,股票代码,原始企业名称,person_id,appln_id,appln_filing_date,appln_auth,appln_kind
0,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,55537629,2007-12-29 00:00:00,EP,A
1,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,56481707,2007-12-29 00:00:00,EP,A
2,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,406838542,2012-06-18 00:00:00,EP,A
3,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,412387138,2013-04-19 00:00:00,EP,A
4,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,414554331,2013-07-26 00:00:00,EP,A
...,...,...,...,...,...,...,...,...,...
2766485,185438472,"QINGDAO HAIER AIR-CONDITIONING ELECTRONIC CO.,...",600690,海尔智家股份有限公司,85438472,568815851,2021-08-20 00:00:00,CN,U
2766486,185438472,"QINGDAO HAIER AIR-CONDITIONING ELECTRONIC CO.,...",600690,海尔智家股份有限公司,85438472,569527621,2021-05-21 00:00:00,CN,U
2766487,185438472,"QINGDAO HAIER AIR-CONDITIONING ELECTRONIC CO.,...",600690,海尔智家股份有限公司,85438472,569545291,2021-08-26 00:00:00,CN,U
2766488,185438472,"QINGDAO HAIER AIR-CONDITIONING ELECTRONIC CO.,...",600690,海尔智家股份有限公司,85438472,569722490,2021-08-31 00:00:00,CN,U


In [234]:
print('专利数量：',len(df_firm_patstat['appln_id'].unique()))
print('公司数量：',len(df_firm_patstat['股票代码'].unique()))

专利数量： 2414991
公司数量： 3932


In [4]:
df_firm_patstat['appln_filing_year']=df_firm_patstat['appln_filing_date'].apply(lambda x:x.year)
dict_appln_filing_year={k:v for k,v in zip(df_firm_patstat['appln_id'],df_firm_patstat['appln_filing_year'])}
dict_appln_filing_date={k:v for k,v in zip(df_firm_patstat['appln_id'],df_firm_patstat['appln_filing_date'])}
dict_appln_filing_date

{55537629: datetime.datetime(2007, 12, 29, 0, 0),
 56481707: datetime.datetime(2007, 12, 29, 0, 0),
 406838542: datetime.datetime(2012, 6, 18, 0, 0),
 412387138: datetime.datetime(2013, 4, 19, 0, 0),
 414554331: datetime.datetime(2013, 7, 26, 0, 0),
 16075341: datetime.datetime(2003, 12, 25, 0, 0),
 328571084: datetime.datetime(2010, 4, 27, 0, 0),
 495792296: datetime.datetime(2017, 12, 8, 0, 0),
 16278451: datetime.datetime(2005, 12, 23, 0, 0),
 56204753: datetime.datetime(2007, 1, 11, 0, 0),
 56264379: datetime.datetime(2008, 2, 8, 0, 0),
 254588: datetime.datetime(2006, 8, 14, 0, 0),
 15868598: datetime.datetime(2001, 6, 13, 0, 0),
 56238657: datetime.datetime(2007, 12, 6, 0, 0),
 16355407: datetime.datetime(2006, 6, 26, 0, 0),
 16348800: datetime.datetime(2006, 3, 29, 0, 0),
 527200941: datetime.datetime(2020, 2, 28, 0, 0),
 16550846: datetime.datetime(1984, 7, 12, 0, 0),
 16644452: datetime.datetime(1986, 12, 8, 0, 0),
 16783416: datetime.datetime(1989, 8, 21, 0, 0),
 17006068: da

In [298]:
sql_string='select t.* from "上市公司匹配patstat_2_title_abstract" t'
df_firm_patstat_title_abstract = pd.read_sql(sql=sql_string, con=connection)
df_firm_patstat_title_abstract['appln_abstract']=df_firm_patstat_title_abstract['appln_abstract'].progress_apply(lambda x:x.read())
df_firm_patstat_title_abstract=df_firm_patstat_title_abstract.fillna('')
df_firm_patstat_title_abstract['appln_title_abstract']=df_firm_patstat_title_abstract['appln_title']+' '+df_firm_patstat_title_abstract['appln_abstract']

df_firm_patstat_title_abstract['title_abstract_len'] = df_firm_patstat_title_abstract['appln_title_abstract'].str.title().progress_apply(lambda x:len(x))
df_firm_patstat_title_abstract['title_abstract_lexicon_count'] = df_firm_patstat_title_abstract['appln_title_abstract'].str.title().progress_apply(lambda x:textstat.lexicon_count(x))
df_firm_patstat_title_abstract['title_abstract_smog_index'] = df_firm_patstat_title_abstract['appln_title_abstract'].str.title().progress_apply(lambda x:textstat.smog_index(x))
df_firm_patstat_title_abstract


100%|███████████████████████████████████████████████████████████████████████| 2382718/2382718 [06:15<00:00, 6350.43it/s]


,appln_id,appln_title,appln_abstract,appln_title_abstract,title_abstract_len,title_abstract_lexicon_count,title_abstract_smog_index
0,2211,"PYRAZOLOPYRIMIDINONE DERIVATIVES, THEIR PREPAR...",The invention discloses a series of pyrazolopy...,"PYRAZOLOPYRIMIDINONE DERIVATIVES, THEIR PREPAR...",731,100,20.6
1,1790,METHOD AND DEVICE FOR MONITORING RECEIVING RAD...,The present invention discloses a method and d...,METHOD AND DEVICE FOR MONITORING RECEIVING RAD...,1367,198,21.9
2,1800,A SYSTEM AND METHOD FOR ADJUSTING AMPLITUDE OF...,The present invention discloses a system and m...,A SYSTEM AND METHOD FOR ADJUSTING AMPLITUDE OF...,1448,217,0.0
3,1815,METHOD AND APPARATUS FOR ELIMINATING NARROW BA...,A device and method for eliminating narrow-ban...,METHOD AND APPARATUS FOR ELIMINATING NARROW BA...,1463,216,26.9
4,2006,A RANDOM ACCESS METHOD FOR USER EQUIPMENT IN T...,The present invention discloses a random acces...,A RANDOM ACCESS METHOD FOR USER EQUIPMENT IN T...,1372,208,0.0
...,...,...,...,...,...,...,...
2382713,573860416,PAGING MECHANISM FOR WIRELESS COMMUNICATION NE...,"Methods, apparatus, and systems for a paging s...",PAGING MECHANISM FOR WIRELESS COMMUNICATION NE...,519,79,0.0
2382714,573860530,ARCHITECTURE FOR RANDOM ACCESS MESSAGING,User equipment may request to communicate with...,ARCHITECTURE FOR RANDOM ACCESS MESSAGING User ...,491,70,14.2
2382715,573878382,RELEASABLE LOCKING MECHANISM FOR A REFRIGERATE...,A releasable locking mechanism for a refrigera...,RELEASABLE LOCKING MECHANISM FOR A REFRIGERATE...,474,80,13.0
2382716,573878426,LINEAR HINGE ASSEMBLY FOR AN APPLIANCE,"A domestic appliance may include a cabinet, a ...",LINEAR HINGE ASSEMBLY FOR AN APPLIANCE A domes...,922,156,13.7


In [273]:
df_citation_normal=df_citation.groupby(['cited_appln_id','cited_appln_filing_year']).agg(num_year=('appln_id','count')).reset_index()
df_citation_year=df_citation_normal.groupby('cited_appln_filing_year').agg(mean_year=('num_year','mean')).reset_index()
df_citation_normal=df_citation_normal.merge(df_citation_year,how='left',left_on='cited_appln_filing_year',right_on='cited_appln_filing_year')
df_citation_normal['citation_normal']=df_citation_normal['num_year']/df_citation_normal['mean_year']
df_citation_normal=df_citation_normal.rename({'cited_appln_id':'appln_id'},axis=1)
df_citation_normal

,appln_id,cited_appln_filing_year,num_year,mean_year,citation_normal
0,1790,2005,3,6.518047,0.460261
1,1800,2005,1,6.518047,0.153420
2,1815,2005,2,6.518047,0.306840
3,2006,2006,6,7.236253,0.829158
4,2495,2006,11,7.236253,1.520124
...,...,...,...,...,...
1182616,569737758,2021,1,1.172718,0.852720
1182617,569910560,2019,1,2.003428,0.499145
1182618,570365664,2021,1,1.172718,0.852720
1182619,573049271,2019,3,2.003428,1.497434


In [250]:

query ='''SELECT t.* FROM "上市公司匹配patstat_2_citation" t '''
df_citation=pd.read_sql(query, connection)
df_citation['cited_appln_filing_year']=df_citation['cited_appln_filing_date'].apply(lambda x: x.year)
df_citation

,pat_publn_id,cited_pat_publn_id,appln_id,cited_appln_id,appln_filing_date,cited_appln_filing_date,cited_appln_filing_year
0,546941131,375751926,546941130,364934659,2020-11-07 00:00:00,2012-02-29 00:00:00,2012
1,441274123,409090713,441274122,409090712,2014-08-27 00:00:00,2012-11-28 00:00:00,2012
2,378011691,275822628,365723367,267427214,2012-03-13 00:00:00,2009-04-17 00:00:00,2009
3,489789520,475193554,475193553,475193553,2015-09-09 00:00:00,2015-09-09 00:00:00,2015
4,538156605,516929217,538156604,516929216,2020-06-24 00:00:00,2019-03-20 00:00:00,2019
...,...,...,...,...,...,...,...
4902719,527714153,510581034,527714152,510581033,2019-12-17 00:00:00,2018-11-20 00:00:00,2018
4902720,384418479,335779927,378107156,328708636,2012-08-01 00:00:00,2010-07-09 00:00:00,2010
4902721,457899772,282776027,457899771,7846426,2016-04-06 00:00:00,2004-10-26 00:00:00,2004
4902722,546942381,330332686,546942380,328713777,2020-11-03 00:00:00,2010-01-26 00:00:00,2010


In [309]:

query ='''SELECT t.* FROM "上市公司匹配patstat_2_citation_3_years" t '''
df_citation_3_years=pd.read_sql(query, connection)
df_citation_3_years=df_citation_3_years.rename({'citation_count':'citation_count_3'},axis=1)
df_citation_3_years

/tmp/ipykernel_101377/2797612905.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_citation_3_years=pd.read_sql(query, connection)


,appln_id,citation_count_3
0,505507126,1
1,525821824,5
2,470502868,7
3,485082841,1
4,497737909,7
...,...,...
920948,479678168,1
920949,418848355,1
920950,536350251,1
920951,446287055,1


In [39]:

query ='''SELECT t.* FROM "上市公司匹配patstat_2_citation_5_years" t '''
df_citation_5_years=pd.read_sql(query, connection)
df_citation_5_years

/tmp/ipykernel_101377/2944147811.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_citation_5_years=pd.read_sql(query, connection)


,appln_id,citation_count
0,409090712,3
1,267427214,8
2,475193553,5
3,516929216,12
4,476361798,14
...,...,...
1074875,274834112,1
1074876,441338129,1
1074877,451958920,1
1074878,539134499,1


In [43]:

query ='''SELECT t.* FROM "上市公司匹配patstat_2_avg_claims" t '''
df_avg_claims=pd.read_sql(query, connection)
df_avg_claims

/tmp/ipykernel_101377/4122219135.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_avg_claims=pd.read_sql(query, connection)


,appln_id,avg_claims,sum_claims
0,28772,0.0,0
1,55395114,0.0,0
2,55752140,1.0,2
3,55877143,0.0,0
4,116763,10.0,10
...,...,...,...
2414986,575102864,0.0,0
2414987,575103428,0.0,0
2414988,574990944,0.0,0
2414989,575046621,0.0,0


In [119]:

query ='''SELECT t.* FROM "上市公司匹配patstat_2_tls231_payment_event" t '''
df_tls231_payment_event=pd.read_sql(query, connection)
df_tls231_payment_event['appln_filing_date']=df_tls231_payment_event['appln_id'].map(dict_appln_filing_date)
df_tls231_payment_event

,event_id,appln_id,event_seq_nr,event_type,event_auth,event_code,event_filing_date,event_publn_date,event_effective_date,event_text,ref_doc_auth,ref_doc_nr,ref_doc_kind,ref_doc_date,ref_doc_text,party_type,party_seq_nr,party_new,party_old,spc_nr,spc_filing_date,spc_patent_expiry_date,spc_extension_date,spc_text,designated_states,extension_states,fee_country,fee_payment_date,fee_renewal_year,fee_text,lapse_country,lapse_date,lapse_text,reinstate_country,reinstate_date,reinstate_text,class_scheme,class_symbol,appln_filing_date
0,8957869,41156144,2,...,LU ...,TA,9999-12-31 00:00:00,1991-06-07,9999-12-31 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,...,9999-12-31 00:00:00,0,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,1978-05-05
1,9396681,53369721,7,...,US ...,DP,9999-12-31 00:00:00,1992-01-07,9999-12-31 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,...,9999-12-31 00:00:00,0,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,1985-09-09
2,25742923,24452854,1,...,IT ...,TA,9999-12-31 00:00:00,1999-10-27,1994-02-25 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,...,9999-12-31 00:00:00,0,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,1980-02-22
3,25742894,24453623,1,...,IT ...,TA,9999-12-31 00:00:00,1999-10-27,1995-02-27 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,...,9999-12-31 00:00:00,0,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,1980-02-25
4,25740751,24438678,1,...,IT ...,TA,9999-12-31 00:00:00,1999-10-27,1996-01-29 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,...,9999-12-31 00:00:00,0,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,1980-01-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139714,697124057,412278958,67,...,EP ...,PGFP,9999-12-31 00:00:00,2021-12-31,9999-12-31 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,NL ...,2021-10-14 00:00:00,9,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,2013-11-08
139715,697124250,412550328,76,...,EP ...,PGFP,9999-12-31 00:00:00,2021-12-31,9999-12-31 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,NL ...,2021-10-14 00:00:00,9,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,2013-11-20
139716,697123867,411818457,65,...,EP ...,PGFP,9999-12-31 00:00:00,2021-12-31,9999-12-31 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,NL ...,2021-10-20 00:00:00,9,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,2013-10-25
139717,697123760,411487583,67,...,EP ...,PGFP,9999-12-31 00:00:00,2021-12-31,9999-12-31 00:00:00,None,...,None,...,9999-12-31 00:00:00,None,None,0,None,None,None,9999-12-31 00:00:00,9999-12-31 00:00:00,9999-12-31 00:00:00,None,None,None,NL ...,2021-10-20 00:00:00,9,None,,9999-12-31 00:00:00,None,,9999-12-31 00:00:00,None,,None,2013-10-21


In [50]:

query ='''SELECT t.* FROM "上市公司匹配patstat_2_tls201" t '''
df_tls201=pd.read_sql(query, connection)
df_tls201

/tmp/ipykernel_101377/4134071221.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tls201=pd.read_sql(query, connection)


,appln_id,appln_auth,appln_nr,appln_kind,appln_filing_date,appln_filing_year,appln_nr_original,ipr_type,receiving_office,internat_appln_id,...,earliest_publn_date,earliest_publn_year,earliest_pat_publn_id,granted,docdb_family_id,inpadoc_family_id,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors
0,1790,EP,05814579,A,2005-11-28 00:00:00,2005,05814579,PI,,7770581,...,2008-08-20 00:00:00,2008,60053,Y,38066895,1790,3,2,1,3
1,1800,EP,05818679,A,2005-12-12 00:00:00,2005,05818679,PI,,7770704,...,2008-08-27 00:00:00,2008,84861,Y,38162530,1800,3,1,1,4
2,1815,EP,05823414,A,2005-12-16 00:00:00,2005,05823414,PI,,7770775,...,2008-08-27 00:00:00,2008,84835,Y,38162538,1815,9,6,1,3
3,2006,EP,06705496,A,2006-01-18 00:00:00,2006,06705496,PI,,7771062,...,2008-10-01 00:00:00,2008,191178,Y,38287229,2006,9,23,1,4
4,2211,EP,06817839,A,2006-11-16 00:00:00,2006,06817839,PI,,7774062,...,2008-08-27 00:00:00,2008,83893,N,38048299,2211,9,9,3,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414986,574993085,US,202117456029,A,2021-11-22 00:00:00,2021,17456029,PI,,0,...,2022-06-30 00:00:00,2022,574993086,N,82118216,574993085,2,0,1,7
2414987,574994908,US,202217585581,A,2022-01-27 00:00:00,2022,17585581,PI,,545758383,...,2022-06-30 00:00:00,2022,574994909,N,74319498,545220117,4,2,1,3
2414988,574998377,US,202117216958,A,2021-03-30 00:00:00,2021,17216958,PI,,0,...,2022-06-30 00:00:00,2022,574998378,N,75455277,549197176,2,1,1,2
2414989,575002042,US,202117644396,A,2021-12-15 00:00:00,2021,17644396,PI,,0,...,2022-06-30 00:00:00,2022,575002043,N,82119734,575002042,2,0,1,3


# 替代性和互补性指标

In [241]:
df_firm_patstat_cpc_AI=pd.read_pickle('data/df_firm_patstat_cpc_AI.pickle')
df_firm_patstat_cpc_AI

,appln_id,cpc_class_symbol,AI_CPC_part1,AI_CPC_part2,block2,AI_CPC,appln_date,appln_filing_year,cpc_class_symbol_AI
4576095,24599,H01M 8/04097,0,0,0,0,2005-12-30,2005,H01M 8/04097_No-AI
4243158,24599,H01M 8/04,0,0,0,0,2005-12-30,2005,H01M 8/04_No-AI
4992327,24599,H01M 8/22,0,0,0,0,2005-12-30,2005,H01M 8/22_No-AI
998957,24599,H01M 8/06,1,0,0,1,2005-12-30,2005,H01M 8/06_AI
1163818,24599,Y02P 70/50,0,0,0,0,2005-12-30,2005,Y02P 70/50_No-AI
...,...,...,...,...,...,...,...,...,...
2496512,575553198,G05D 1/0219,1,0,0,1,2021-12-27,2021,G05D 1/0219_AI
333345,575553198,G05D2201/0208,0,0,0,0,2021-12-27,2021,G05D2201/0208_No-AI
1914269,575554198,G10L 15/32,1,0,1,1,2021-12-31,2021,G10L 15/32_AI
4909513,575554198,G10L 15/02,1,0,1,1,2021-12-31,2021,G10L 15/02_AI


In [242]:
df_firm_patstat_CPC=df_firm_patstat[['股票代码','appln_id']].merge(df_firm_patstat_cpc_AI)
df_firm_patstat_CPC=df_firm_patstat_CPC.groupby(['股票代码','appln_filing_year','cpc_class_symbol_AI']).agg(
                                    cpc_class_symbol_count=('cpc_class_symbol_AI','count'),
                                                                                   ).reset_index()
df_firm_patstat_CPC=df_firm_patstat_CPC[df_firm_patstat_CPC['cpc_class_symbol_AI'].str.contains('_No-AI')]
df_firm_patstat_CPC['cpc_class_symbol_AI_cum'] = df_firm_patstat_CPC.groupby(['股票代码','cpc_class_symbol_AI'])['cpc_class_symbol_count'].cumsum()

df_firm_patstat_CPC

,股票代码,appln_filing_year,cpc_class_symbol_AI,cpc_class_symbol_count,cpc_class_symbol_AI_cum
0,000001,2019,G06F 3/167_No-AI,1,1
1,000001,2019,G06F 11/261_No-AI,1,1
2,000001,2019,G06F 11/3612_No-AI,1,1
3,000001,2019,G06F 11/3664_No-AI,1,1
4,000001,2019,G06F 11/3684_No-AI,1,1
...,...,...,...,...,...
110347,872925,2020,G10L 25/18_No-AI,1,1
110348,872925,2020,G10L 25/24_No-AI,1,1
110349,872925,2020,H04L 67/10_No-AI,1,1
110350,872925,2020,H04L 67/125_No-AI,1,1


In [243]:
df_firm_patstat_CPC_bg=df_firm_patstat.merge(
    df_firm_patstat_cpc_AI[['appln_id','appln_filing_year','cpc_class_symbol_AI']],how='inner',left_on=['appln_id','appln_filing_year'],right_on=['appln_id','appln_filing_year']).merge(
    df_firm_patstat_CPC[['appln_filing_year','cpc_class_symbol_AI','cpc_class_symbol_AI_cum']],how='inner',left_on=['appln_filing_year','cpc_class_symbol_AI'],right_on=['appln_filing_year','cpc_class_symbol_AI']).groupby(['股票代码','appln_id']).agg(
    cpc_class_symbol_AI_cum=('cpc_class_symbol_AI_cum','sum')).reset_index()
df_firm_patstat_CPC_bg

,股票代码,appln_id,cpc_class_symbol_AI_cum
0,000001,519303535,18
1,000001,519304119,19
2,000001,519919568,22
3,000001,520193829,20
4,000001,520426090,132
...,...,...,...
40553,839790,560886584,19
40554,871694,531873472,87
40555,872541,520848178,364
40556,872925,539457501,221


In [244]:
cpc_class_symbol_AI_list=df_firm_patstat_cpc_AI.groupby('appln_id').agg(
    cpc_class_symbol=('cpc_class_symbol',lambda x:list(x.str[:4]))
    ).reset_index()
cpc_class_symbol_AI_list['cpc_class_symbol_len']=cpc_class_symbol_AI_list['cpc_class_symbol'].apply(lambda x:len(set(x)))
cpc_class_symbol_AI_list['cpc_class_symbol_entropy']=cpc_class_symbol_AI_list['cpc_class_symbol'].apply(lambda x:entropy(x))
cpc_class_symbol_AI_list['cpc_class_symbol_gini']=cpc_class_symbol_AI_list['cpc_class_symbol'].apply(lambda x:gini(x))
# cpc_class_symbol_AI_list=cpc_class_symbol_AI_list.merge(cpc_class_symbol_AI_list_AI[['appln_id','cpc_class_symbol_AI_entropy','cpc_class_symbol_AI_gini']],left_on='appln_id',right_on='appln_id')
cpc_class_symbol_AI_list

,appln_id,cpc_class_symbol,cpc_class_symbol_len,cpc_class_symbol_entropy,cpc_class_symbol_gini
0,24599,"[H01M, H01M, H01M, H01M, Y02P, H01M, H01M, H01...",3,0.865857,0.484848
1,39826,"[H01M, Y02E, H01M, H01M, H01M, H01M, H01M, H01...",2,0.439497,0.409091
2,1514227,"[F16J, B29C, B29C, Y10S, B29C]",3,1.370951,0.266667
3,1932606,"[B01J, C01B, H01M, Y02E, B01J, B01J, C01B, H01...",5,1.947703,0.377778
4,2641533,"[H04M, G10L, G10L]",2,0.918296,0.166667
...,...,...,...,...,...
43660,575543179,"[G10L, G10L, H04L]",2,0.918296,0.166667
43661,575543848,"[G10L, G10L, G10L, G10L, G10L]",1,0.000000,0.000000
43662,575543988,"[G06T, G06N, G06T, G06K, G06T, G06T, G06T, G06...",3,1.370951,0.266667
43663,575553198,"[G05D, G05D]",1,0.000000,0.000000


# 关联数据

In [245]:
df_firm_patstat_cpc_AI_group=pd.read_pickle('data/df_firm_patstat_cpc_AI_group.pickle')
df_firm_patstat_cpc_AI_group['appln_filing_year']=df_firm_patstat_cpc_AI_group['appln_id'].map(dict_appln_filing_year)
df_firm_patstat_cpc_AI_group=df_firm_patstat_cpc_AI_group[df_firm_patstat_cpc_AI_group['complementarity_mean'].notnull()]
df_firm_patstat_cpc_AI_group

AI_CPC,appln_id,No-AI,AI,complementarity_mean,substitution_mean,complementarity_sum,substitution_sum,appln_filing_year
0,24599,"[H01M 8/04097_No-AI, H01M 8/04_No-AI, H01M...",[H01M 8/06_AI],0.425000,0.728652,4.250000,7.286519,2005
1,39826,"[Y02E 60/50_No-AI, H01M 8/04201_No-AI, H01M...",[H01M 8/06_AI],0.472727,0.762472,4.727273,7.624718,2006
3,1932606,"[B01J2219/2465_No-AI, C01B2203/0822_No-AI, H01...",[H01M 8/06_AI],0.539216,0.896139,9.166667,15.234365,2001
4,2641533,[H04M 1/271_No-AI],"[G10L 15/26_AI, G10L2015/027_AI]",0.111111,0.670820,0.222222,1.341641,2003
5,2706776,"[G06Q 30/0603_No-AI, G06Q 30/0277_No-AI, G06...","[G06Q 10/00_AI, G06Q 10/02_AI]",0.250000,0.756874,2.500000,7.568736,2005
...,...,...,...,...,...,...,...,...
43660,575543179,[H04L 12/282_No-AI],"[G10L2015/223_AI, G10L 15/22_AI]",0.013234,0.702513,0.026468,1.405026,2021
43661,575543848,"[G10L 19/0018_No-AI, G10L 13/02_No-AI, G10L ...","[G10L 17/02_AI, G10L 17/04_AI]",0.000802,0.272191,0.004812,1.633149,2021
43662,575543988,"[G06T 7/11_No-AI, G06K 9/6256_No-AI, G06T2...","[G06T2207/20081_AI, G06N 3/04_AI, G06T2207/2...",0.033813,0.576151,0.811511,13.827632,2020
43663,575553198,[G05D2201/0208_No-AI],[G05D 1/0219_AI],0.000962,0.411177,0.000962,0.411177,2021


In [246]:
def calculate_4_digital(x):
    new_x=[i[0:4] for i in x]
    return len(set(new_x))

In [311]:

df_patent_information=df_firm_patstat.merge(
    df_firm_patstat_cpc_AI_group,how='inner',left_on=['appln_id','appln_filing_year'],right_on=['appln_id','appln_filing_year']).merge(
    df_citation_5_years,how='left',left_on=['appln_id'],right_on=['appln_id']).merge(
    df_citation_3_years,how='left',left_on=['appln_id'],right_on=['appln_id']).merge(
    df_citation_normal[['appln_id', 'citation_normal']],how='left',left_on=['appln_id'],right_on=['appln_id']).merge(
    df_avg_claims,how='left',left_on=['appln_id'],right_on=['appln_id']).merge(
    df_tls201[['appln_id','docdb_family_size','nb_citing_docdb_fam','nb_applicants','nb_inventors']],how='left',left_on=['appln_id'],right_on=['appln_id']).merge(
    cpc_class_symbol_AI_list[['appln_id','cpc_class_symbol_len','cpc_class_symbol_entropy','cpc_class_symbol_gini']],how='left',left_on=['appln_id'],right_on=['appln_id']).merge(
    df_firm_patstat_CPC_bg,how='left',left_on=['股票代码','appln_id'],right_on=['股票代码','appln_id']).merge(
    df_firm_patstat_title_abstract[['appln_id', 'title_abstract_len', 'title_abstract_lexicon_count', 'title_abstract_smog_index']],how='left',left_on=['appln_id'],right_on=['appln_id'])
df_patent_information['renew']=np.where(df_patent_information['appln_id'].isin(df_tls231_payment_event['appln_id']),1,0)
df_patent_information['patent_age']=2022-df_patent_information['appln_filing_year']
fillpad_list=['cpc_class_symbol_AI_cum']
for col in fillpad_list:
    df_patent_information[col]=df_patent_information.groupby('股票代码')[col].transform(lambda x: x.fillna(method='ffill')) #前一个的值来填充
fillna_list=['citation_count','citation_count_3','avg_claims','sum_claims','docdb_family_size','nb_citing_docdb_fam','nb_applicants',
             'nb_inventors','cpc_class_symbol_AI_cum','citation_normal']
df_patent_information[fillna_list]=df_patent_information[fillna_list].fillna(0)
df_patent_information['len_No_AI']=df_patent_information['No-AI'].apply(lambda x:len(x))
df_patent_information['len_AI']=df_patent_information['AI'].apply(lambda x:len(x))
df_patent_information['len_No_AI_4']=df_patent_information['No-AI'].apply(lambda x:calculate_4_digital(x))
df_patent_information['len_AI_4']=df_patent_information['AI'].apply(lambda x:calculate_4_digital(x))
df_patent_information['CN_A']=np.where((df_patent_information['appln_auth']=='CN')&(df_patent_information['appln_kind'].str.contains('A')),1,0)
# df_patent_information=df_patent_information.fillna(0)
df_patent_information=df_patent_information.dropna()
df_patent_information=df_patent_information[df_patent_information['len_No_AI']>0]
df_patent_information.rename({'股票代码':'id_org','appln_filing_year':'year'},axis=1).to_csv('data/df_patent_information.csv')
df_patent_information


,psn_id,psn_name,股票代码,原始企业名称,person_id,appln_id,appln_filing_date,appln_auth,appln_kind,appln_filing_year,No-AI,AI,complementarity_mean,substitution_mean,complementarity_sum,substitution_sum,citation_count,citation_count_3,citation_normal,avg_claims,sum_claims,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors,cpc_class_symbol_len,cpc_class_symbol_entropy,cpc_class_symbol_gini,cpc_class_symbol_AI_cum,title_abstract_len,title_abstract_lexicon_count,title_abstract_smog_index,renew,patent_age,len_No_AI,len_AI,len_No_AI_4,len_AI_4,CN_A
0,1311438,ARCSOFT,688088,虹软科技股份有限公司,5269123,54371889,2001-09-28 00:00:00,US,A,2001,"[G06T2207/20096_No-AI, G06T 7/12_No-AI, G06T...",[G06T2210/62_AI],0.166667,0.750000,0.666667,3.000000,8.0,8.0,6.178161,3.5,7,1,20,1,1,1,0.000000,0.000000,4.0,1023.0,164.0,14.8,1,21,4,1,1,1,0
1,9110624,FOXCONN,601138,富士康工业互联网股份有限公司,5255507,378519244,2011-05-23 00:00:00,CN,A,2011,[G05D2201/0216_No-AI],"[G05D 1/0282_AI, G05D 1/0265_AI]",0.150000,0.558114,0.300000,1.116228,9.0,4.0,4.751308,0.0,0,3,33,2,4,1,0.000000,0.000000,3.0,1088.0,183.0,16.9,0,11,1,2,1,1,1
2,9110624,FOXCONN,601138,富士康工业互联网股份有限公司,5212189,374801276,2011-06-24 00:00:00,US,A,2011,[B64C 39/024_No-AI],[G05D 1/0033_AI],0.100000,0.000000,0.100000,0.000000,6.0,1.0,7.602093,0.0,0,2,36,4,3,2,1.000000,0.000000,4.0,603.0,99.0,0.0,0,11,1,1,1,1,0
3,9110624,FOXCONN,601138,富士康工业互联网股份有限公司,5368462,451852839,2014-07-30 00:00:00,TW,A,2014,"[G07C2209/63_No-AI, B60R 25/24_No-AI, G01S ...","[B62D 15/0285_AI, B60W 30/06_AI, G05D 1/02...",0.035772,0.646651,0.536585,9.699762,0.0,0.0,0.000000,0.0,0,4,6,1,2,6,2.500000,0.166667,11.0,864.0,130.0,12.0,0,8,5,3,3,3,0
4,9110624,FOXCONN,601138,富士康工业互联网股份有限公司,5368462,490613654,2016-05-20 00:00:00,TW,A,2016,"[B25J 11/0005_No-AI, B25J 11/008_No-AI, B25J...",[B25J 9/161_AI],0.004926,0.149517,0.019704,0.598067,0.0,0.0,0.000000,0.0,0,3,6,1,1,1,0.000000,0.000000,7.0,549.0,78.0,14.8,0,6,4,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51071,184424822,"STATE GRID CO., LTD.",300902,国安达股份有限公司,84424822,568052710,2021-12-03 00:00:00,CN,A,2021,"[G06F2111/04_No-AI, G06F 30/27_No-AI]","[G06Q 50/06_AI, G06Q 40/04_AI, G06N 3/006_AI]",0.013555,0.550394,0.081328,3.302364,1.0,1.0,0.852720,0.0,0,1,1,2,17,3,1.521928,0.133333,1036.0,1448.0,209.0,29.5,0,1,2,3,1,2,1
51072,184424822,"STATE GRID CO., LTD.",300902,国安达股份有限公司,84424822,568765031,2021-11-17 00:00:00,CN,A,2021,"[G06K 9/6224_No-AI, G06N 3/0445_No-AI, G06...",[G06N 3/08_AI],0.393969,0.432559,1.181906,1.297676,0.0,0.0,0.000000,0.0,0,1,0,2,3,2,0.811278,0.250000,8709.0,966.0,128.0,24.5,0,1,3,1,2,1,1
51073,184424822,"STATE GRID CO., LTD.",300902,国安达股份有限公司,84424822,569522593,2022-02-23 00:00:00,CN,A,2022,"[G06Q 30/0202_No-AI, G06Q 50/06_No-AI]",[G06Q 10/04_AI],0.211297,0.443212,0.422594,0.886423,1.0,1.0,0.909091,0.0,0,1,1,3,17,1,0.000000,0.000000,2690.0,1374.0,201.0,0.0,0,0,2,1,1,1,1
51074,185114360,SOUTHERN POWER GRID ELECTRIC POWER SCIENCE AND...,688248,南方电网电力科技股份有限公司,85114360,568053970,2021-12-07 00:00:00,CN,A,2021,"[G06K 9/6201_No-AI, G06T 7/73_No-AI, G06N ...","[G06T2207/20084_AI, G06N 3/08_AI, G06T2207/2...",0.079457,0.556900,1.430221,10.024202,0.0,0.0,0.000000,0.0,0,1,0,1,7,3,1.224394,0.370370,8108.0,1186.0,186.0,24.2,0,1,6,3,3,2,1


In [317]:
df_patent_information_des=df_patent_information[(df_patent_information['appln_filing_year']>=2010)&(df_patent_information['appln_filing_year']<=2022)]
print('专利数量：',len(df_patent_information_des['appln_id'].unique()))
print('专利数量：',len(df_patent_information_des['股票代码'].unique()))

专利数量： 37679
专利数量： 1024


In [314]:
df_patent_information.appln_kind.unique()

array(['A ', 'W ', 'U ', 'T '], dtype=object)

In [294]:
df_patent_information_group=df_patent_information.groupby(['股票代码','appln_filing_year']).agg(
    AI_count=('appln_id','count'),
    complementarity_mean=('complementarity_mean','mean'),
    substitution_mean=('substitution_mean','mean'),
    complementarity_sum=('complementarity_sum','mean'),
    substitution_sum=('substitution_sum','mean'),
    citation_count=('citation_count','sum'),
    citation_normal=('citation_normal','sum'),
    avg_claims=('avg_claims','mean'),
    sum_claims=('sum_claims','sum'),
    docdb_family_size=('docdb_family_size','sum'),
    nb_citing_docdb_fam=('nb_citing_docdb_fam','sum'),
    nb_applicants=('nb_applicants','sum'),
    nb_inventors=('nb_inventors','sum'),
    cpc_class_symbol_len=('cpc_class_symbol_len','sum'),
    cpc_class_symbol_entropy=('cpc_class_symbol_entropy','mean'),
    cpc_class_symbol_gini=('cpc_class_symbol_gini', 'mean'),
    cpc_class_symbol_AI_cum=('cpc_class_symbol_AI_cum', 'sum'),
    title_string_len=('title_string_len','mean'),
    title_lexicon_count=('title_lexicon_count', 'mean'),
    title_smog_index=('title_smog_index', 'mean'),
    abstract_string_len=('abstract_string_len','mean'),
    abstract_lexicon_count=('abstract_lexicon_count', 'mean'),
    abstract_smog_index=('abstract_smog_index','mean'),
    renew=('renew','sum'),
    ).reset_index().rename({'股票代码':'id_org','appln_filing_year':'year'},axis=1).to_csv('data/df_patent_information_group.csv')

df_patent_information_group

In [92]:
df_gre=df_patent_information_group.merge(
    df_tobin,how='left',left_on=['id_org','year'],right_on=['id_org','year']).merge(
                  df_RD_cost[['id_org','year','RD_cost']],left_on=['id_org','year'],right_on=['id_org','year'],how='left').merge(
                  df_size[['id_org','year','size','lnSale']],left_on=['id_org','year'],right_on=['id_org','year'],how='left').merge(
                  df_controls,left_on=['id_org','year'],right_on=['id_org','year'],how='left').merge(
                  df_zhuanyongxing,left_on=['id_org','year'],right_on=['id_org','year'],how='left').merge(
                  df_jijindu,left_on=['id_org','year'],right_on=['id_org','year'],how='left').merge(
                  df_HHI,left_on=['id_org','year'],right_on=['id_org','year'],how='left')
# 使用向后填充
df_gre['Sicda_str'] = df_gre.groupby('id_org')['Sicda_str'].transform(lambda x: x.fillna(method='bfill'))
# 对于仍然存在的缺失值，使用向前填充
df_gre['Sicda_str'] = df_gre.groupby('id_org')['Sicda_str'].transform(lambda x: x.fillna(method='ffill'))
df_gre['EquityNature']=df_gre.groupby('id_org')['EquityNature'].transform(lambda x: x.fillna(method='bfill'))
df_gre['EquityNature']=df_gre.groupby('id_org')['EquityNature'].transform(lambda x: x.fillna(method='ffill'))

# df_gre = df_gre[~df_gre['Sicda_str'].isin(['J66','J67','J68','J69',])]
# # df_gre = df_gre[~df_gre['Sicda_str'].str.contains('F|G|H|J|K|L',na=False)]
# # df_gre = df_gre.groupby('id_org').filter(lambda x: (x['patent_count'] > 0).any())
# # df_gre = df_gre.groupby('id_org').filter(lambda x: (x['paper_count'] > 0).any())
# fillna_list=['citation_count']
# df_gre[fillna_list]=df_gre[fillna_list].fillna(0)
df_gre

,id_org,year,AI_count,complementarity_mean,substitution_mean,complementarity_sum,substitution_sum,citation_count,sum_claims,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors,Stknme,IsSTPT,Estbyear,Sicda_str,Nindnme51,营业利润,EquityNature,tobin_A,tobin_B,tobin_C,tobin_D,市值A,市值B,营业利润增长率A,流动资产合计,资产总额,固定资产净额,F100401A,age,lnage,tobin,RD_cost,size,lnSale,员工人数1,员工人数2,RDPerson,RDPersonRatio,Shrcr1,研发费用率,RDSpendSumRatio,RDInvestRatio,员工人数,资产专用性1,资产专用性2,资产专用性3,创新倾向,成长性,生产效率,市场扩张倾向,组织结构稳定性,资本密度,创新倾向得分,成长性得分,生产效率得分,市场扩张倾向得分,组织结构稳定性得分,资本密度得分,企业战略激进度,企业战略,防御型企业战略,分析型企业战略,进攻型企业战略,赫芬达尔指数,勒纳指数
0,000001,2019,12,0.013139,0.318407,0.086970,0.849104,8.0,0,12,8,12,29,平安银行,否,1987.0,J66,银行业,36289000000,外资,1.001585,1.004628,1.001585,1.004628,3945314301097.417,3945314354357.1,-.477707,None,3939070000000,11092000000,1.019955,32.0,3.496508,1.003106,NaN,29.00196577076728,NaN,NaN,34253.0,None,None,49.565,None,None,None,34253.0,0.003923,0.001107,-0.00086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000001,2020,48,0.078280,0.485712,0.409684,2.751184,8.0,0,48,8,48,136,平安银行,否,1987.0,J66,银行业,36909000000,外资,1.002502,1.005070,1.002502,1.005070,4479693363630.959,4479693457949.32,-.291016,None,4468514000000,10893000000,1.030706,33.0,3.526361,1.003786,NaN,29.12807703080536,NaN,NaN,38097.0,None,None,49.565,None,None,None,38097.0,0.0033,0.000862,-0.000697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000001,2021,97,0.147443,0.448044,0.451204,2.116834,9.0,0,97,9,97,235,平安银行,否,1987.0,J66,银行业,45985000000,外资,0.984631,0.987313,0.984631,0.987313,4845743076359.293,4845741531903.04,-.364784,None,4921380000000,11974000000,.808731,34.0,3.555348,0.985972,NaN,29.22461009490829,NaN,NaN,40651.0,None,None,49.565,None,None,None,40651.0,0.003612,0.001179,-0.00123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000012,2020,2,0.262950,0.524982,0.788850,1.574946,0.0,0,2,0,4,2,南玻A,否,1984.0,C30,玻璃及玻璃制品业,1111875682,民营,1.371674,1.485753,1.372403,1.486543,24529521152.9779,24542560576.05239,-.656681,4639837067,17882914898,9145644569,2.134698,36.0,3.610918,1.429093,4.197420e+08,23.60711161901976,23.090819,10558.0,10558.0,170,1.61,15.1883,.037938,4.07,6.86,10558.0,0.681607,0.063732,-0.062008,0.106803,0.019025,0.009894,0.084423,0.0635,0.511418,2,2,2,1,1,0,8,分析型企业战略,0,1,0,0.202207,0.217959
4,000016,2014,4,0.027500,0.184340,0.065000,0.450011,29.0,0,4,26,4,5,深康佳A,否,1980.0,C39,日用电子器具制造业,-149612758.11,国企,1.063569,1.086308,1.092188,1.115538,17846012002.88646,18326215930.68301,None,12870313630.13,16779359276.65,1783695548.92,1.678155,34.0,3.555348,1.089401,2.190000e+08,23.54341535350496,23.689749,NaN,18776.0,None,None,21.7508,None,None,None,18776.0,0.13807,0.020717,-0.044227,0.017897,-0.029153,0.009667,0.159673,0.040749,0.106303,0,0,1,2,0,3,6,防御型企业战略,1,0,0,0.094515,-0.021194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2810,839790,2018,1,0.666667,0.417626,1.333333,0.835252,4.0,0,1,4,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016731,0.007098,-0.004658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.141982,None
2811,871694,2019,1,0.001746,0.221591,0.017456,2.215911,0.0,0,1,0,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2812,872541,2019,1,0.024273,0.471428,0.364090,7.071423,3.0,0,1,3,1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [93]:
df_gre=df_gre.merge(df_dynamic,how='left',left_on=['id_org','year'],right_on=['id_org','year'])
df_gre=df_gre.merge(df_dynamic_CV,how='left',left_on=['id_org','year'],right_on=['id_org','year'])


In [95]:
# df_gre=df_gre.fillna(0)
df_gre=df_gre.interpolate(method='linear')
df_gre=df_gre.rename({'组织结构稳定性':'Stability','行业调整后的环境不确定性':'Uncertainty','营业利润':'Profit','资产总额':'Assets',
               '流动资产合计':'Current_Assets','固定资产净额':'Fixed_Assets','员工人数':'Employee','市值A':'Market_value'},axis=1)
df_gre.to_csv('data/df_gre.csv',encoding='utf-8-sig',index=None)
df_gre

,id_org,year,AI_count,complementarity_mean,substitution_mean,complementarity_sum,substitution_sum,citation_count,sum_claims,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors,Stknme,IsSTPT,Estbyear,Sicda_str,Nindnme51,Profit,EquityNature,tobin_A,tobin_B,tobin_C,tobin_D,Market_value,市值B,营业利润增长率A,Current_Assets,Assets,Fixed_Assets,F100401A,age,lnage,tobin,RD_cost,size,lnSale,员工人数1,员工人数2,RDPerson,RDPersonRatio,Shrcr1,研发费用率,RDSpendSumRatio,RDInvestRatio,Employee,资产专用性1,资产专用性2,资产专用性3,创新倾向,成长性,生产效率,市场扩张倾向,Stability,资本密度,创新倾向得分,成长性得分,生产效率得分,市场扩张倾向得分,组织结构稳定性得分,资本密度得分,企业战略激进度,企业战略,防御型企业战略,分析型企业战略,进攻型企业战略,赫芬达尔指数,勒纳指数,current_asset_turnover,fixed_asset_turnover,total_asset_turnover,captal_asset_turnover,income,RD_cost1,captal_outcome,CV_RD_cost,CV_captal_outcome
0,000001,2019,12,0.013139,0.318407,0.086970,0.849104,8.0,0,12,8,12,29,平安银行,否,1987.00,J66,银行业,36289000000,外资,1.001585,1.004628,1.001585,1.004628,3945314301097.417,3945314354357.1,-.477707,None,3939070000000,11092000000,1.019955,32.0,3.496508,1.003106,NaN,29.00196577076728,NaN,NaN,34253.0,None,None,49.565,None,None,None,34253.0,0.003923,0.001107,-0.00086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.508179,0.033902,NaN,NaN,NaN,0.355300,NaN,0.0
1,000001,2020,48,0.078280,0.485712,0.409684,2.751184,8.0,0,48,8,48,136,平安银行,否,1987.00,J66,银行业,36909000000,外资,1.002502,1.005070,1.002502,1.005070,4479693363630.959,4479693457949.32,-.291016,None,4468514000000,10893000000,1.030706,33.0,3.526361,1.003786,NaN,29.12807703080536,NaN,NaN,38097.0,None,None,49.565,None,None,None,38097.0,0.0033,0.000862,-0.000697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.060642,0.033569,NaN,NaN,NaN,0.349000,NaN,0.0
2,000001,2021,97,0.147443,0.448044,0.451204,2.116834,9.0,0,97,9,97,235,平安银行,否,1987.00,J66,银行业,45985000000,外资,0.984631,0.987313,0.984631,0.987313,4845743076359.293,4845741531903.04,-.364784,None,4921380000000,11974000000,.808731,34.0,3.555348,0.985972,NaN,29.22461009490829,NaN,NaN,40651.0,None,None,49.565,None,None,None,40651.0,0.003612,0.001179,-0.00123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.274215,0.032319,NaN,NaN,NaN,0.661400,NaN,0.0
3,000012,2020,2,0.262950,0.524982,0.788850,1.574946,0.0,0,2,0,4,2,南玻A,否,1984.00,C30,玻璃及玻璃制品业,1111875682,民营,1.371674,1.485753,1.372403,1.486543,24529521152.9779,24542560576.05239,-.656681,4639837067,17882914898,9145644569,2.134698,36.0,3.610918,1.429093,4.197420e+08,23.60711161901976,23.090819,10558.000000,10558.0,170,1.61,15.1883,.037938,4.07,6.86,10558.0,0.681607,0.063732,-0.062008,0.106803,0.019025,0.009894,0.084423,0.0635,0.511418,2,2,2,1,1,0,8,分析型企业战略,0,1,0,0.202207,0.217959,2.282711,1.137345,0.592781,13.001730,1.067125,0.040484,0.111077,0.0,0.0
4,000016,2014,4,0.027500,0.184340,0.065000,0.450011,29.0,0,4,26,4,5,深康佳A,否,1980.00,C39,日用电子器具制造业,-149612758.11,国企,1.063569,1.086308,1.092188,1.115538,17846012002.88646,18326215930.68301,None,12870313630.13,16779359276.65,1783695548.92,1.678155,34.0,3.555348,1.089401,2.190000e+08,23.54341535350496,23.689749,10991.454545,18776.0,None,None,21.7508,None,None,None,18776.0,0.13807,0.020717,-0.044227,0.017897,-0.029153,0.009667,0.159673,0.040749,0.106303,0,0,1,2,0,3,6,防御型企业战略,1,0,0,0.094515,-0.021194,1.525505,10.613390,1.185232,13.672642,1.942349,0.040317,0.075096,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2810,839790,2018,1,0.666667,0.417626,1.333333,0.835252,4.0,0,1,4,1,2,NaN,NaN,2011.75,NaN,NaN,NaN,NaN,1.467951,1.475239,2.088797,2.096156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2.302585,1.782036,1.035682e+07,NaN,19.604944,9011.740741,1545.4,NaN,NaN,NaN,NaN,NaN,NaN,1545.4,0.016731,0.007098,-0.004658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [89]:
data = pd.DataFrame({'Individual':['A','A','A','B','B',],'Year':[2000,2001,2002,2000,2001,],'A': [1, np.nan, 3, np.nan, 2], 'B': [np.nan, 2, np.nan, 2, np.nan]})
data

,Individual,Year,A,B
0,A,2000,1.0,NaN
1,A,2001,NaN,2.0
2,A,2002,3.0,NaN
3,B,2000,NaN,2.0
4,B,2001,2.0,NaN


In [90]:
data.interpolate(method='linear')

,Individual,Year,A,B
0,A,2000,1.0,NaN
1,A,2001,2.0,2.0
2,A,2002,3.0,2.0
3,B,2000,2.5,2.0
4,B,2001,2.0,2.0
